## Simulating observations with MUSTANG-2

MUSTANG-2 is a bolometric array on the [Green Bank Telescope](https://en.wikipedia.org/wiki/Green_Bank_Telescope). In this notebook we simulate an observation of the Whirlpool Galaxy (M51).

In [ ]:
import maria
from maria.io import fetch

input_map = maria.map.load("/users/tom/maria/data/maps/crab_nebula.fits", nu=93e9)

input_map.plot()
print(input_map)

In [ ]:
from maria import Planner

planner = Planner(target=input_map, site="green_bank", el_bounds=(60, 90))
plan = planner.generate_plan(total_duration=900,  # in seconds
                             scan_pattern="daisy",
                             scan_options={"radius": input_map.width.deg / 3},
                             sample_rate=50)

plan.plot()
print(plan)

In [ ]:
instrument = maria.get_instrument("MUSTANG-2")
print(instrument)
instrument.plot()

In [ ]:
sim = maria.Simulation(
    instrument,
    plan=plan,
    site="green_bank",
    map=input_map,
    atmosphere="2d",
)

print(sim)

In [ ]:
tod = sim.run()
tod.plot()

In [ ]:
from maria.mappers import BinMapper

mapper = BinMapper(
    center=input_map.center,
    frame="ra_dec",
    width=10 / 60,
    height=10 / 60,
    resolution=0.05 / 60,
    tod_preprocessing={
        "window": {"name": "hamming"},
        "remove_modes": {"modes_to_remove": [0]},
        "remove_spline": {"knot_spacing": 30, "remove_el_gradient": True},
    },
    map_postprocessing={
        "gaussian_filter": {"sigma": 1},
        "median_filter": {"size": 1},
    },
    units="uK_RJ",
)

mapper.add_tods(tod)
output_map = mapper.run()

In [ ]:
output_map.plot()
output_map.to_fits("/tmp/simulated_mustang_map.fits")